# [Module 4.4] Personalize 캠페인과 실시간 상호 작용 하기

이 노트북은 사용자의 실시간 행동에 반응하는 기능을 추가하는 과정을 안내합니다. 영화를 탐색하는 동안 사용자의 의도가 변경되면, 해당 동작에 따라 수정된 추천 영화 목록들이 표시됩니다.
이 노트북은 다음과 같은 작업을 합니다.
- 이벤트 추적기 생성
- 이전 노트북에서 실행한 처음 추천 결과 보기
- 새로운 영화를 클릭했다고 가정하고, 이벤트 트랙커 업데이트 후에 두 번째 추천 결과 보기
- 다른 새로운 영화를 클릭했다고 가정하고, 이벤트 트랙커 업데이트 후에 세 번째 추천 결과 보기

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid
from random import randint

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [2]:
%store -r


생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

In [3]:
# suffix = str(np.random.uniform())[4:9]

In [4]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')

## 이벤트 추적기 생성

추천 시스템이 실시간 이벤트에 응답하기 전에 이벤트 추적기(Event Tracker)가 필요합니다. 아래 코드 셀에서 이벤트 추적기 하나를 생성하고 이 실습에서 계속 사용하겠습니다. 이벤트 추적기 이름을 `MovieClickTracker`로 부여하였지만, 여러분들이 자유롭게 지정하실 수 있습니다.<br>
위의 추적 ID가 표시되며, 이 ID는 변수에 할당되었으므로 추가 조치가 필요하지 않습니다. 아래 코드 셀을 실행하면 나중에 추천 영화 목록을 출력할 수 있도록 합니다.

In [5]:
TRACKING_ID = '50d0f462-527d-4cb4-9252-1e77c9476cf1'

#### help 함수(영화 타이틀, 장르 리턴)



In [6]:
# First load items into memory
items_all = pd.read_csv('./ml-1m/movies.dat',sep='::', encoding='latin1',names=['ITEM_ID', 'TITLE', 'GENRE'],)
items=items_all.copy()
items['to_keep'] = items['ITEM_ID'].apply(lambda x:x in unique_items)
items=items[items['to_keep']]
del items['to_keep']
print(items.tail())

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)
    movie_title=items[items['ITEM_ID']==movie_id]['TITLE']
    return (movie_title.tolist())

def get_movie_genre(movie_id):
    """
    Takes in an ID, returns a genre
    """
    movie_id = int(movie_id)
    movie_genre=items[items['ITEM_ID']==movie_id]['GENRE']
    return (movie_genre.tolist())

# Help 함수
def rec_item_list(item_list):
    recommendation_title_list = []
    recommendation_genre_list = []
    # recommendation_id_list=[]
    for item in item_list:
        title = get_movie_title(item['itemId'])
        genre = get_movie_genre(item['itemId'])
        recommendation_title_list.append(title)
        recommendation_genre_list.append(genre)    
        
    recommendations_df = pd.DataFrame(data={'OriginalRecs':recommendation_title_list,
                                            'Genre': recommendation_genre_list})
    return recommendations_df                                                



      ITEM_ID                       TITLE           GENRE
3878     3948     Meet the Parents (2000)          Comedy
3879     3949  Requiem for a Dream (2000)           Drama
3880     3950            Tigerland (2000)           Drama
3881     3951     Two Family House (2000)           Drama
3882     3952       Contender, The (2000)  Drama|Thriller


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


      ITEM_ID                       TITLE           GENRE
3878     3948     Meet the Parents (2000)          Comedy
3879     3949  Requiem for a Dream (2000)           Drama
3880     3950            Tigerland (2000)           Drama
3881     3951     Two Family House (2000)           Drama
3882     3952       Contender, The (2000)  Drama|Thriller


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


## 특정 유저 선택

In [19]:
# user_id = 4375 # 특정 유저 선택, 결과 확인 후에 추후에 주석 표시하여 랜더 유저 선택할 수 있습니다.
user_id = 1500 # 특정 유저 선택, 결과 확인 후에 추후에 주석 표시하여 랜더 유저 선택할 수 있습니다.
print("USER: {}".format(user_id))



USER: 1500


## 선택된 유저의 과거 영화 레이팅 정보

In [20]:
from utils import get_rich_dataset

# 학습에 사용된 warm_train 데이터 셋 로딩
df_warm_train = pd.read_csv(warm_train_interaction_filename)
# item 정보 로딩
item_meta = pd.read_csv('./ml-1m/movies.dat',sep='::', encoding='latin1',names=['ITEM_ID', 'TITLE', 'GENRE'],)

def get_recentVeiw(user_id, df_rich, num_history):
    '''
    최신 리스트 및 추천 아이템을 제공 함
    '''
    history_items = df_rich[df_rich['USER_ID']==user_id].tail(num_history)    
    return history_items


# 영황 타이틀, 장르를 포함한 정보 리턴
df_warm_train_rich = get_rich_dataset(df_warm_train, item_meta)
df_warm_train_rich = df_warm_train_rich.sort_values('TIMESTAMP').copy()
# df_warm_train_rich

user_history = get_recentVeiw(user_id, df_warm_train_rich, num_history=20)
user_history





/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


,USER_ID,ITEM_ID,TITLE,GENRE,TIMESTAMP,DATE
8745,1500,2918,Ferris Bueller's Day Off (1986),Comedy,988252171,2001-04-26 02:29:31
187858,1500,3039,Trading Places (1983),Comedy,988252199,2001-04-26 02:29:59
164918,1500,3253,Wayne's World (1992),Comedy,988252224,2001-04-26 02:30:24
184983,1500,3608,Pee-wee's Big Adventure (1985),Comedy,988252224,2001-04-26 02:30:24
153144,1500,3481,High Fidelity (2000),Comedy,988252224,2001-04-26 02:30:24
242507,1500,934,Father of the Bride (1950),Comedy,988252252,2001-04-26 02:30:52
171468,1500,1257,Better Off Dead... (1985),Comedy,988252269,2001-04-26 02:31:09
36550,1500,3108,"Fisher King, The (1991)",Comedy|Drama|Romance,988252311,2001-04-26 02:31:51
317382,1500,2915,Risky Business (1983),Comedy,988252339,2001-04-26 02:32:19
239661,1500,441,Dazed and Confused (1993),Comedy,988252339,2001-04-26 02:32:19


## 첫번째 추천

In [22]:
# 'gsmoon'
user_personalization_campaign_arn = 'arn:aws:personalize:ap-northeast-2:057716757052:campaign/Movielens-user-personalization-campaign-60498'


In [23]:

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = user_personalization_campaign_arn,    
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

rec_exp_df = rec_item_list(item_list)
rec_exp_df

Recommendations for user:  1500


,OriginalRecs,Genre
0,[Pulp Fiction (1994)],[Crime|Drama]
1,[Gladiator (2000)],[Action|Drama]
2,[Mission: Impossible 2 (2000)],[Action|Thriller]
3,[Erin Brockovich (2000)],[Drama]
4,[Good Will Hunting (1997)],[Drama]
5,[Citizen Kane (1941)],[Drama]
6,[High Fidelity (2000)],[Comedy]
7,[American Psycho (2000)],[Comedy|Horror|Thriller]
8,"[Shawshank Redemption, The (1994)]",[Drama]
9,[Magnolia (1999)],[Drama]


## 사용자 행동 시뮬레이션

아래 코드 셀은 특정 item과 상호 작용하는 사용자를 시뮬레이트하는 코드 샘플을 제공하며, 시작할 때와 다른 추천 목록을 얻습니다.

아래 ```send_movie_click(USER_ID, ITEM_ID)``` 는 다음과 같은 작업을 하여 실시간 이벤트를 반영 합니다.

- 유저에 대한 세션이 없으면 SessionID를 생성하고, 있으면 이 유저의 SessionID를 불러온다
- event를 정의하고 event에 item_id를 할당한다
- event를 json 형태로 바꾼다
- 위에서 생성한 event tranker의 put_event에 위 json파일을 인자로 넘겨서 호출한다.

#### Help 함수

In [24]:
session_dict = {}

def send_movie_click(user_id, ITEM_ID, verbose=False):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[user_id]
    except:
        session_dict[user_id] = str(uuid.uuid1())
        session_ID = session_dict[user_id]
   
    value=randint(0,5)
    
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    "eventValue": value
    }
    event_json = json.dumps(event)
    
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID, # 이벤트트래커에서 생성한 아이디
    userId= user_id,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'RATING',
        'properties': event_json
        }])
            
                    
    if verbose:
        print("trankingId: ", TRACKING_ID)
        print("USER_ID: ", user_id)
        print("sessionID: ", session_ID)
        print("timestamp: ", int(time.time()))
        print("properties: ", event_json)
        

    
# Help 함수

def rec_item_rich_list(item_list, clicked_title, clicked_genre):
    recommendation_title_list = []
    recommendation_genre_list = []

    for item in item_list:
        title = get_movie_title(item['itemId'])
        genre = get_movie_genre(item['itemId'])
        recommendation_title_list.append(title)
        recommendation_genre_list.append(genre)    

    title_df = pd.DataFrame(recommendation_title_list, columns=[clicked_title])
    genre_df = pd.DataFrame(recommendation_genre_list, columns=[clicked_genre])        
    recommendations_df = title_df.join(genre_df)

    return recommendations_df                                                    

def get_new_recommend(user_id,campaign_arn, recommendations_df, title, genre):    
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id),
    )

    item_list = get_recommendations_response['itemList']
    new_recs_df = rec_item_rich_list(item_list,title,  genre)
    new_recommendations_df = pd.concat([recommendations_df,new_recs_df ], axis=1)

    print("Recommendations for user: ", user_id)
    return new_recommendations_df
    

## Movie 선택 및 두번째 추천 리스트

이번에는 토이 스토리를 클릭했다고 가정하겠습니다.<br>
이후에 추천의 변화를 보시죠.<br>
"movie_title: Toy Story (1995), movie_genre: Animation|Children's|Comedy" 선택에 따른 새로운 추천의 변화
- 아래 테이블의 결과를 보는 방법은 첫번째, 두번째 컬럼은 오지널 추천 영화의 제목 및 장르 입니다. <br>
세번째의 컬럼의 맨위는 "클릭한 영화 제목" 이고 이하는 추천 리스트 입니다. 그리고 네번째 컬럼의 맨위는 세번째 컬럼의 각 해당하는 영화의 장르 입니다.

**새로운 추천이 시간에 대한 입력값이 들어가기에 추천 결과가 실행시마다 상이할 수 있습니다.**


In [25]:
pd.options.display.max_columns = 10

import time

# Pick a movie, we will use ID 100 or (Postino, Il (The Postman) (1994),)
##########################
# 영화 선택
##########################
movie_to_click = 95 # movie_title: Toy Story (1995), movie_genre: Action|Thriller
# movie_to_click = 2 # movie_title: Toy Story (1995), movie_genre: Animation|Children's|Comedy
##########################

movie_title_clicked = get_movie_title(movie_to_click)
movie_genre_clicked = get_movie_genre(movie_to_click)
print("user_id: {}, movie_id_clicked: {}, movie_title: {}, movie_genre: {}".format(user_id, movie_to_click, movie_title_clicked[0], movie_genre_clicked[0]))
# 이벤트 추적기 put_event 호출
print("current user_id: ", user_id)
send_movie_click(user_id=str(user_id), ITEM_ID=movie_to_click, verbose=True)
time.sleep(10) # 이벤트 트랙커가 업데이트 되게 잠시 기다립니다.
# # 새로운 추천 결과
rec_exp_df = get_new_recommend(user_id,user_personalization_campaign_arn, rec_exp_df,movie_title_clicked, movie_genre_clicked )    

rec_exp_df


user_id: 1500, movie_id_clicked: 95, movie_title: Broken Arrow (1996), movie_genre: Action|Thriller
current user_id:  1500
trankingId:  50d0f462-527d-4cb4-9252-1e77c9476cf1
USER_ID:  1500
sessionID:  e4407868-6921-11eb-bb5b-0f8f3f28037f
timestamp:  1612687985
properties:  {"itemId": "95", "eventValue": 5}
Recommendations for user:  1500


,OriginalRecs,Genre,"(Broken Arrow (1996),)","(Action|Thriller,)"
0,[Pulp Fiction (1994)],[Crime|Drama],Pulp Fiction (1994),Crime|Drama
1,[Gladiator (2000)],[Action|Drama],Good Will Hunting (1997),Drama
2,[Mission: Impossible 2 (2000)],[Action|Thriller],Boys Don't Cry (1999),Drama
3,[Erin Brockovich (2000)],[Drama],"Shawshank Redemption, The (1994)",Drama
4,[Good Will Hunting (1997)],[Drama],Amadeus (1984),Drama
5,[Citizen Kane (1941)],[Drama],Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi
6,[High Fidelity (2000)],[Comedy],Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
7,[American Psycho (2000)],[Comedy|Horror|Thriller],Gladiator (2000),Action|Drama
8,"[Shawshank Redemption, The (1994)]",[Drama],Braveheart (1995),Action|Drama|War
9,[Magnolia (1999)],[Drama],Citizen Kane (1941),Drama


## 또 다른 Movie 선택 및 세번째 추천 리스트

그 다음에는 알라딘을 클릭한 것을 시뮬레이션 합니다.
"movie_title: Aladdin (1992), movie_genre: Animation|Children's|Comedy|Musical" 선택에 따른 새로운 추천의 변화
- 아래 테이블의 결과를 보는 방법은 첫번째, 두번째, 세번째, 네번째는 위에서 확인한 것과 동일 합니다.<br>
다섯번째의 컬럼의 맨위는 이번에 "클릭한 영화 제목" 이고 이하는 추천 리스트 입니다. 그리고 여섯번째 컬럼의 맨위는 다섯번째 컬럼의 각 해당하는 영화의 장르 입니다.

**새로운 추천이 시간에 대한 입력값이 들어가기에 추천 결과가 실행시마다 상이할 수 있습니다.**


In [26]:


# Pick a movie, we will use ID 100 or (Postino, Il (The Postman) (1994),)
##########################
# 영화 선택
##########################
# movie_to_click = 588 # movie_title: Aladdin (1992), movie_genre: Animation|Children's|Comedy|Musical
movie_to_click = 1687 # movie_title: Aladdin (1992), movie_genre: Animation|Children's|Comedy|Musical
##########################
movie_title_clicked = get_movie_title(movie_to_click)
movie_genre_clicked = get_movie_genre(movie_to_click)
print("user_id: {}, movie_id_clicked: {}, movie_title: {}, movie_genre: {}".format(user_id, movie_to_click, 
                                                                                   movie_title_clicked[0], movie_genre_clicked[0]))
# 이벤트 추적기 put_event 호출
send_movie_click(user_id=str(user_id), ITEM_ID=movie_to_click)
time.sleep(10) # 이벤트 트랙커가 업데이트 되게 잠시 기다립니다.
rec_exp_df = get_new_recommend(user_id,user_personalization_campaign_arn, rec_exp_df,
                                       movie_title_clicked, movie_genre_clicked )    
rec_exp_df

user_id: 1500, movie_id_clicked: 1687, movie_title: Jackal, The (1997), movie_genre: Action|Thriller
Recommendations for user:  1500


,OriginalRecs,Genre,"(Broken Arrow (1996),)","(Action|Thriller,)","(Jackal, The (1997),)","(Action|Thriller,)"
0,[Pulp Fiction (1994)],[Crime|Drama],Pulp Fiction (1994),Crime|Drama,Pulp Fiction (1994),Crime|Drama
1,[Gladiator (2000)],[Action|Drama],Good Will Hunting (1997),Drama,Braveheart (1995),Action|Drama|War
2,[Mission: Impossible 2 (2000)],[Action|Thriller],Boys Don't Cry (1999),Drama,Good Will Hunting (1997),Drama
3,[Erin Brockovich (2000)],[Drama],"Shawshank Redemption, The (1994)",Drama,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
4,[Good Will Hunting (1997)],[Drama],Amadeus (1984),Drama,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi
5,[Citizen Kane (1941)],[Drama],Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi,"Shawshank Redemption, The (1994)",Drama
6,[High Fidelity (2000)],[Comedy],Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller,"Sixth Sense, The (1999)",Thriller
7,[American Psycho (2000)],[Comedy|Horror|Thriller],Gladiator (2000),Action|Drama,Amadeus (1984),Drama
8,"[Shawshank Redemption, The (1994)]",[Drama],Braveheart (1995),Action|Drama|War,Raiders of the Lost Ark (1981),Action|Adventure
9,[Magnolia (1999)],[Drama],Citizen Kane (1941),Drama,Gladiator (2000),Action|Drama


## 3번째 시뮬레이션

In [27]:
import time

# Pick a movie, we will use ID 100 or (Postino, Il (The Postman) (1994),)
##########################
# 영화 선택
##########################
# movie_to_click = 1907 # movie_title: Mulan (1998), movie_genre: Animation|Children's
movie_to_click = 786 # movie_title: Mulan (1998), movie_genre: Animation|Children's
##########################
movie_title_clicked = get_movie_title(movie_to_click)
movie_genre_clicked = get_movie_genre(movie_to_click)
print("user_id: {}, movie_id_clicked: {}, movie_title: {}, movie_genre: {}".format(user_id, movie_to_click, 
                                                                                   movie_title_clicked[0], movie_genre_clicked[0]))
# 이벤트 추적기 put_event 호출
send_movie_click(user_id=str(user_id), ITEM_ID=movie_to_click)
time.sleep(10) # 이벤트 트랙커가 업데이트 되게 잠시 기다립니다.
rec_exp_df = get_new_recommend(user_id,user_personalization_campaign_arn, rec_exp_df,
                                       movie_title_clicked, movie_genre_clicked )    
rec_exp_df

user_id: 1500, movie_id_clicked: 786, movie_title: Eraser (1996), movie_genre: Action|Thriller
Recommendations for user:  1500


,OriginalRecs,Genre,"(Broken Arrow (1996),)","(Action|Thriller,)","(Jackal, The (1997),)","(Action|Thriller,)","(Eraser (1996),)","(Action|Thriller,)"
0,[Pulp Fiction (1994)],[Crime|Drama],Pulp Fiction (1994),Crime|Drama,Pulp Fiction (1994),Crime|Drama,"Lost World: Jurassic Park, The (1997)",Action|Adventure|Sci-Fi|Thriller
1,[Gladiator (2000)],[Action|Drama],Good Will Hunting (1997),Drama,Braveheart (1995),Action|Drama|War,Armageddon (1998),Action|Adventure|Sci-Fi|Thriller
2,[Mission: Impossible 2 (2000)],[Action|Thriller],Boys Don't Cry (1999),Drama,Good Will Hunting (1997),Drama,Braveheart (1995),Action|Drama|War
3,[Erin Brockovich (2000)],[Drama],"Shawshank Redemption, The (1994)",Drama,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller,Star Trek: Insurrection (1998),Action|Sci-Fi
4,[Good Will Hunting (1997)],[Drama],Amadeus (1984),Drama,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi,Air Force One (1997),Action|Thriller
5,[Citizen Kane (1941)],[Drama],Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi,"Shawshank Redemption, The (1994)",Drama,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi
6,[High Fidelity (2000)],[Comedy],Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller,"Sixth Sense, The (1999)",Thriller,Con Air (1997),Action|Adventure|Thriller
7,[American Psycho (2000)],[Comedy|Horror|Thriller],Gladiator (2000),Action|Drama,Amadeus (1984),Drama,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
8,"[Shawshank Redemption, The (1994)]",[Drama],Braveheart (1995),Action|Drama|War,Raiders of the Lost Ark (1981),Action|Adventure,Twister (1996),Action|Adventure|Romance|Thriller
9,[Magnolia (1999)],[Drama],Citizen Kane (1941),Drama,Gladiator (2000),Action|Drama,Alien: Resurrection (1997),Action|Horror|Sci-Fi


## 결론

사용자가 상호 작용하는 영화를 변경하여 추천 사항이 변경되었음을 알 수 있습니다. 이 시스템은 사용자가 item 모음과 상호 작용하는 모든 응용 프로그램으로 확장할 수 있습니다. 이러한 도구는 언제든지 사용 가능한 데이터로 가능한 것을 풀다운하여 탐색할 수 있습니다.

In [ ]:
%store event_tracker_arn